In [1]:
#importing required packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split

In [20]:
df = pd.read_csv('trainms.csv')
# df.columns

In [77]:
df['work_interfere']=df['work_interfere'].fillna("Maybe")
df['self_employed']=df['self_employed'].fillna("Dont Know")

In [97]:
ax = df[df['comments'].isnull() == False]
print(len(ax))
no_comments = ax[ax['treatment']=='No'].comments
print(len(no_comments))
for i in range(len(no_comments)):
    print(no_comments.iloc[i])

127
51
I'm not on my company's health insurance which could be part of the reason I answered Don't know to so many questions.
In Russia we have mandatory medical insurance. Every employer must pay 3.6% of every employee's salary to the insurance fund like a tax. Everyone gets free healthcare at public clinics and hospitals. Some types of healthcare including mental health are still funded from the government budget but that doesn't matter it's still FREE!However this is Russia :D I don't know much about what actually happens in mental hospitals here except that a lot of young people with male in their passports use these to avoid military draft and doctors seem to help them.
I'd be more worried about coworkers and workplace culture than the employer--they're probably legally obligated to do some things but reputation among people I work with is something else. For instance I've heard people make snide remarks about men taking paternity leave I don't want to know what they'd say about m

In [ ]:
df[df['self_employed'].isnull()]

In [22]:
temp = df.Gender
temp = temp.replace(to_replace=["A little about you","something kinda male?","Enby","fluid","Genderqueer","Trans-female","Nah","Androgyne","Agender","male leaning androgynous","Trans woman","Neuter","Female (trans)","queer","non-binary"],value="T")
temp = temp.replace(["Female","female","Cis Female","F","Woman","f","Femake","Female ","woman","cis-female/femme","Female (cis)"],'F')
temp = temp.replace(["M","Male","male","m","Male-ish","maile","Cis Male","Mal","Male ","Make","Guy (-ish) ^_^","Man","msle","Mail","cis male"],'M')
temp.unique()
df.Gender = temp

In [ ]:
temp_df = df.drop(columns=['state','comments','s.no','Timestamp'])
le = preprocessing.LabelEncoder()
temp_df = temp_df.apply(le.fit_transform)
temp_df.plot('Country','treatment', 'scatter', alpha=0.01, figsize=(10,8))

In [54]:
to_drop_columns = ['treatment','state','comments','s.no','Timestamp']+['Country','supervisor','coworkers','phys_health_interview','anonymity','self_employed','remote_work','tech_company','obs_consequence','phys_health_consequence','seek_help','Gender']

In [55]:
features = df.drop(columns=to_drop_columns)
target = df.treatment

le = preprocessing.LabelEncoder()
features = features.apply(le.fit_transform)
labels,uniques = pd.factorize(target)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)


In [75]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=-1, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:

# for i in range(len(clf.estimators_)):
i=0
estimator = clf.estimators_[i]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X_train.columns,
                class_names = ["Yes" if i==0 else "No" for i in np.unique(y_train)],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpdf', 'tree.dot', '-o', 'tree'+str(i)+'.pdf', '-Gdpi=600'])

print("done")

# # Display in jupyter notebook
# from IPython.display import Image
# Image(filename = 'tree.png')

In [76]:
#Verify predictions
train_predicted = clf.predict(X_train)

accuracy = accuracy_score(y_train,train_predicted)
print(accuracy)
# print(confusion_matrix(labels,train_predicted))
pd.crosstab(y_train, train_predicted, rownames=['Actual'], colnames=['Predicted'])
list(zip(X_train, clf.feature_importances_))
# xxxx = colmns
# colmns= []
# for i,j in importances:
#     if j<0.02:
#         colmns.append(i)
# print(colmns)
#                Predicted
#                  No Yes
#  Acutal    No
#            Yes

0.9761194029850746


[('Age', 0.13020557934412458),
 ('family_history', 0.05551588272472909),
 ('work_interfere', 0.38708967956489426),
 ('no_employees', 0.06841388603822231),
 ('benefits', 0.052714101302093044),
 ('care_options', 0.08521899407010927),
 ('wellness_program', 0.03309754535734645),
 ('leave', 0.058274791053761765),
 ('mental_health_consequence', 0.050995748814747124),
 ('mental_health_interview', 0.018771051114062257),
 ('mental_vs_physical', 0.05970274061590982)]

In [74]:
test_predicted = clf.predict(X_test)
accuracy = accuracy_score(y_test,test_predicted)
print(accuracy)

0.8090909090909091


In [ ]:
from pdpbox import pdp
from plotnine import *

# x=pd.DataFrame({'cols':X_train.columns, 'imp':clf.feature_importances_}).sort_values('imp', ascending=False)
# plot_fi(x[:10])


In [ ]:
test = pd.read_csv('testms.csv')
temp = test.Age
temp = temp.replace(to_replace=-1,value=22)
test.Age = temp

test['work_interfere']=test['work_interfere'].fillna("Maybe")
test['self_employed']=test['self_employed'].fillna("Dont Know")

In [ ]:
#Validate test gender
temp = test.Gender
temp = temp.replace(to_replace=["Male","male","m","Malr","Male ","Cis Man"],value='M')
temp = temp.replace(to_replace=["female","F","Female","Woman","femail","f"],value='F')
temp = temp.replace(to_replace=["p","Female (trans)","ostensibly male, unsure what that really means"],value='T')
test.Gender = temp

In [ ]:
#predict testing data
col = to_drop_columns
col.remove('treatment')
test_features = test.drop(columns=col)
le = preprocessing.LabelEncoder()
test_features = test_features.apply(le.fit_transform)

pred = clf.predict(test_features)

In [ ]:
# Submission generation
index = [str(i) for i in range(1,len(pred)+1)]
values = ['Yes' if val==0 else 'No' for i,val in enumerate(pred)]
submission_result = pd.DataFrame(values, index=index,columns=['treatment'])
submission_result.index.name = "s.no"

file_name="submission.csv"
submission_result.to_csv(file_name, sep=',')